## 데이터 집계와 그룹 연산
- 하나 이상의 키(함수, 배열, DataFrame 컬럼 이름)을 이용해서 pandas 객체를 여러 조각으로 나누는 방법
- 합계, 평균, 표준편차 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
- 정규화, 선형회귀 , 등급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법
- 변위치 분석과 다른 통계 집단 분석을 수행하는 방법

- **기본 Pandas 함수 에서 제거**
- **시계열 데이터의 집계의 경우 groupby = 리샘플링방법(resampling)**
- **사용방법    ==>     데이터.groupby(['열이름']).적용함수()**

![그룹연산예시](imgs/그룹연산예시.png)

## 데이터 선언

In [1]:
import numpy as np
import pandas as pd

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.267266,-0.254217
1,a,two,2.697491,-0.865592
2,b,one,-2.177184,1.285283
3,b,two,0.861819,0.223845
4,a,one,0.024136,0.205877


## groupby 객체

In [2]:
# grouped 변수는 GroupBy객체이고, 그룹연산을 위해 필요한 정보를 가지고 있는 것.
grouped = df['data1'].groupby(df['key1'])
grouped

# 그룹별 평균 구하기.
grouped.mean()

key1
a    1.329631
b   -0.657682
Name: data1, dtype: float64

## groupby 옵션

In [3]:
# mean
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

# size
sizes = df['data1'].groupby([df['key1'], df['key2']]).size()
sizes

key1  key2
a     one     0.645701
      two     2.697491
b     one    -2.177184
      two     0.861819
Name: data1, dtype: float64

key1  key2
a     one     2
      two     1
b     one     1
      two     1
Name: data1, dtype: int64

## Unstack 계층해제하기

In [4]:
# unstack으로 계층해제하기
means.unstack()
print(type(means.unstack())) # DataFrame or Series 형태 반환

key2,one,two
key1,,
a,0.645701,2.697491
b,-2.177184,0.861819


<class 'pandas.core.frame.DataFrame'>


## Groupby 그룹 순회하기

In [5]:
# 그룹간 순회하기
for name, group in df.groupby('key1'):
    print('name',name)
    print(group,'\n')
    
    
# 그룹순회 - dict형으로 가져오기
pieces = dict(list(df.groupby('key1')))
pieces

name a
  key1 key2     data1     data2
0    a  one  1.267266 -0.254217
1    a  two  2.697491 -0.865592
4    a  one  0.024136  0.205877 

name b
  key1 key2     data1     data2
2    b  one -2.177184  1.285283
3    b  two  0.861819  0.223845 



{'a':   key1 key2     data1     data2
 0    a  one  1.267266 -0.254217
 1    a  two  2.697491 -0.865592
 4    a  one  0.024136  0.205877,
 'b':   key1 key2     data1     data2
 2    b  one -2.177184  1.285283
 3    b  two  0.861819  0.223845}

## 컬럼이나 컬럼의 일부만 선택하기

In [19]:
# 여러 컬럼 선택해서 (key1, key2) data2 groupby
df_ = df.groupby(['key1','key2'])[['data2']].mean()
df_

# 컬럼의 일부만 선택해서 groupby
df['data1'].groupby(df['key1']).mean()

data2
key1 key2          
a    one  -0.024170
     two  -0.865592
b    one   1.285283
     two   0.223845

key1
a    1.329631
b   -0.657682
Name: data1, dtype: float64

## Groupby 후에 다시 인덱스로 올리기

In [20]:
# groupby 후에 다시 인덱스로 올리기
df_.reset_index(level=['key2'], inplace = True)
df_

,key2,data2
key1,,
a,one,-0.024170
a,two,-0.865592
b,one,1.285283
b,two,0.223845


## 사전과 Series에서 그룹핑하기
- 그룹 정보는 배열이 아닌 형태로 존재하기도 한다.
- dict 형태로 존재할 때 키값으로 데이터 모으고 value로 컬럼 데이터

In [7]:
people = pd.DataFrame(np.random.randn(5,5), 
                      columns=['a','b','c','d','e'],
                     index = ['Joe','Steve','Wes','Jim','Travis'])

# 2행 (1번,2번 컬럼) nan
people.iloc[2:3, [1,2]] = np.nan

# dict
mapping = {'a':'red','b':'red','c':'blue','d':'blue',
           'e':'red','e':'red','f':'orange'}
mapping
print('groupby 기준 dictionary')

# people
people
print('people data')

# groupyby - dict
by_column = people.groupby(mapping, axis=1)
by_column.sum()
print('groupby 계산결과')

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

groupby 기준 dictionary


,a,b,c,d,e
Joe,1.405834,-1.138996,0.788227,2.692907,0.128181
Steve,-0.256916,-1.209834,-1.665202,-1.128335,-0.521307
Wes,0.942263,NaN,NaN,-0.967487,1.487368
Jim,-0.165298,1.461942,0.114100,-0.083197,0.231558
Travis,-0.076581,-2.150225,1.329538,0.513535,0.982293


people data


,blue,red
Joe,3.481135,0.395019
Steve,-2.793537,-1.988057
Wes,-0.967487,2.429631
Jim,0.030903,1.528202
Travis,1.843074,-1.244513


groupby 계산결과


## 함수로 그룹핑하기
- 색인값으로 row 인덱스를 쓴다는 것 명심

In [8]:
# 이름의 길이별로 합계 통계
people.groupby(len).sum() # len 이라고 쓰면 그 기준이 알아서 index의 len이 됨.

# 이름의 길이별로 합계 통계 2
ls = [3,5,3,3,6]
people.groupby(ls).sum()

# len 과 key_list 분류로 이어짐
key_list=['one','one','one','two','two']
people.groupby([len, key_list]).min()

,a,b,c,d,e
3,2.182800,0.322946,0.902328,1.642223,1.847107
5,-0.256916,-1.209834,-1.665202,-1.128335,-0.521307
6,-0.076581,-2.150225,1.329538,0.513535,0.982293


,a,b,c,d,e
3,2.182800,0.322946,0.902328,1.642223,1.847107
5,-0.256916,-1.209834,-1.665202,-1.128335,-0.521307
6,-0.076581,-2.150225,1.329538,0.513535,0.982293


a         b         c         d         e
3 one  0.942263 -1.138996  0.788227 -0.967487  0.128181
  two -0.165298  1.461942  0.114100 -0.083197  0.231558
5 one -0.256916 -1.209834 -1.665202 -1.128335 -0.521307
6 two -0.076581 -2.150225  1.329538  0.513535  0.982293

## 데이터 집계
- 데이터 집계는 배열로부터 스칼라값을 만들어내는 모든 데이터 변환 작업을 말함.
- 집계함수는 일반적으로 좀더 느리게 동작하는데, 중간 데이터를 생성하는 과정에서 함수 호출이나 데이터 정렬 같은 오버헤드가발생하기 때문
- count, sum, mean, median, std, var, min, max, prod(NA가 아닌 값들의 곱), first(NA가 아닌 값들 중 첫번째값), last(NA가 아닌 값들 중 마지막값)
- quantile은 groupby를 위해서 구현되진않았고, Series메서드로 사용가능.

In [9]:
df
print('원래 데이터')

grouped = df.groupby('key1')

grouped['data1'].quantile(0.9)

,key1,key2,data1,data2
0,a,one,1.267266,-0.254217
1,a,two,2.697491,-0.865592
2,b,one,-2.177184,1.285283
3,b,two,0.861819,0.223845
4,a,one,0.024136,0.205877


원래 데이터


key1
a    2.411446
b    0.557919
Name: data1, dtype: float64

## 자신만의 데이터 집계함수 사용 
- 배열의 aggregate나 agg 메서드에 해당 함수 넘기기

In [10]:
# 최대에서 최소빼기
def max_sub_min(arr):
    return arr.max() - arr.min()

# agg 함수 - 사용자정의함수
grouped.agg(max_sub_min)

# aggregate 함수 - mean
grouped.aggregate('mean')

,data1,data2
key1,,
a,2.673355,1.071469
b,3.039004,1.061438


,data1,data2
key1,,
a,1.329631,-0.304644
b,-0.657682,0.754564


## map함수, apply 함수, applymap 함수

### 1. map함수 
- DataFrame이 아닌 Series 타입에서만 사용 가능.
    - Sereis란 = 인덱스(index) + 값(value)

### 2. apply함수
- apply함수는 Series 객체에서도 사용가능하고, DataFrame 객체에서도 사용가능하다는 
- **apply 함수는 DataFrame의 행/열 기반으로 작동 / applymap은 DataFrame에서 요소별로 작동**
- **가장 일반적인 GroupBy 메서드의 목적. apply 하기 위해서**

 
### 3. applymap 함수
- DataFrame에서 요소별로 작동함

### 요약
- map은 Series형에서의 각각의 데이터에 대한 접근.
- apply가 DataFrame의 행, 열에 대한 함수 적용 결과
- applymap DataFrame에 접근하되 각각의 데이터에 접근


In [11]:
### map 함수는 Series 형에대해서만 적용 가능.

frame = pd.DataFrame(np.random.randn(4, 3), 
                  columns=list('bde'), 
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

f_map = lambda x: '%.2f' % x

# series라 작동가능
frame['e'].map(f_map)


,b,d,e
Utah,0.164546,0.042872,-0.148056
Ohio,-1.821897,-0.712336,-0.607178
Texas,1.700927,0.067268,1.148575
Oregon,-0.148342,1.781940,-0.936732


Utah      -0.15
Ohio      -0.61
Texas      1.15
Oregon    -0.94
Name: e, dtype: object

In [12]:
### apply 함수
frame

# f 함수 정의
f_apply = lambda x: x.max() - x.min()

# 세로 열에 대해 f함수
frame.apply(f_apply, axis=0) 

# 가로 행에 대해 f함수
frame.apply(f_apply, axis=1) 


,b,d,e
Utah,0.164546,0.042872,-0.148056
Ohio,-1.821897,-0.712336,-0.607178
Texas,1.700927,0.067268,1.148575
Oregon,-0.148342,1.781940,-0.936732


b    3.522824
d    2.494276
e    2.085308
dtype: float64

Utah      0.312602
Ohio      1.214718
Texas     1.633659
Oregon    2.718672
dtype: float64

In [13]:
### applymap 함수
# apply가 DataFrame의 행, 열로 접근, applymap 은 그 각각의 데이터에 접근
frame

f_applymap = lambda x: '%.3f' % x
frame.applymap(f_applymap)

,b,d,e
Utah,0.164546,0.042872,-0.148056
Ohio,-1.821897,-0.712336,-0.607178
Texas,1.700927,0.067268,1.148575
Oregon,-0.148342,1.781940,-0.936732


,b,d,e
Utah,0.165,0.043,-0.148
Ohio,-1.822,-0.712,-0.607
Texas,1.701,0.067,1.149
Oregon,-0.148,1.782,-0.937


## 변위치 분석과 버킷 분석
- cut함수와 qcut 메서드 이용. 선택한 크기만큼 표본 변위치에 따라 데이터를 나눔
- N등분 및, group apply, unstack

In [14]:
frame2 = pd.DataFrame({'data1':np.random.randn(100),
                       'data2':np.random.randn(100)})

# 4등분 하기
quartiles = pd.cut(frame2.data1, 4)
quartiles

0        (0.62, 2.166]
1        (0.62, 2.166]
2     (-2.471, -0.926]
3       (-0.926, 0.62]
4     (-2.471, -0.926]
            ...       
95      (-0.926, 0.62]
96      (-0.926, 0.62]
97      (-0.926, 0.62]
98      (-0.926, 0.62]
99       (0.62, 2.166]
Name: data1, Length: 100, dtype: category
Categories (4, interval[float64]): [(-4.023, -2.471] < (-2.471, -0.926] < (-0.926, 0.62] < (0.62, 2.166]]

In [15]:
# 4등분 groupby 결과
grouped = frame2.data2.groupby(quartiles)
grouped.mean()

# min, max, count, mean 구해서 리턴
def get_stats(group):
    return {'min':group.min(), 'max':group.max(),
             'count':group.count(), 'mean':group.mean()}

grouped_all = grouped.apply(get_stats)
grouped_all

# unstack 으로 group형 풀기
grouped_all.unstack()

data1
(-4.023, -2.471]   -0.118484
(-2.471, -0.926]   -0.026854
(-0.926, 0.62]      0.016675
(0.62, 2.166]       0.191457
Name: data2, dtype: float64

data1                  
(-4.023, -2.471]  min      -0.421797
                  max       0.184829
                  count     2.000000
                  mean     -0.118484
(-2.471, -0.926]  min      -1.157092
                  max       1.406712
                  count    22.000000
                  mean     -0.026854
(-0.926, 0.62]    min      -1.925883
                  max       1.791961
                  count    50.000000
                  mean      0.016675
(0.62, 2.166]     min      -1.313934
                  max       2.221130
                  count    26.000000
                  mean      0.191457
Name: data2, dtype: float64

,min,max,count,mean
data1,,,,
"(-4.023, -2.471]",-0.421797,0.184829,2.0,-0.118484
"(-2.471, -0.926]",-1.157092,1.406712,22.0,-0.026854
"(-0.926, 0.62]",-1.925883,1.791961,50.0,0.016675
"(0.62, 2.166]",-1.313934,2.221130,26.0,0.191457
